## Introduction

The purpose of the following implementation was to replicate the research result presented in the Schmid's 1994 paper *Part-of-speech tagging with neural networks* (Schmid, 1994). More specifically, I re-implemented the whole data pipeline to prepared the same format learning data used in the original paper. As for the neural network modeling, since I relied on Keras package to build and train the neural network, I had to make some simplification to the original training architecture. With the architecture specified above, my implementation successfully replicated the original training result. It basically obtained the same model performance as the original implementation.

The following discussion divided into two part: first, I walk through the implementation, explaining how I replicated the original data pipeline and specified the neural network architecture. Second, I discuss my training result and compare it to the original model.

## Implementation Walkthrough

## 1. Data Preparation

- **data source and data splitting**

The training corpus used in this implementation was the sample corpus of Penn Treebank Corpus provided by the python package NLTK. This sample corpus contained about 3.9K sentences and 10K words. However, not every word in the sample corpus was tagged with one of the 36 part-of-speech tag defined by Penn Treebank Corpus. Therefore, after filtering out those words not tagged with a valid part-of-speech tag, there are about 8.2K words can be used in part-of-speech tagger training. 

Here I used 80% of words as training data, and among the training data, 90% of words were used to build model and 10% of word were used as validation set. Testing data accounted for 20% of all words. 

In [25]:
import nltk
import pandas as pd
nltk.download('treebank')
nltk.download('universal_tagset')
from nltk.corpus import treebank
from nltk.util import ngrams
from scipy.stats import entropy
import numpy as np
from sklearn.feature_extraction import DictVectorizer

[nltk_data] Downloading package treebank to C:\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to C:\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [249]:
POS_TAG =['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','SYM','TO','UH','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB']
OPEN_CLASS_POS=['JJ','JJR','JJS','RB','RBR','RBS','NNS','NN','VBZ','VB','VBG','VBP','VBN','VBD']

sentences = treebank.tagged_sents()
filter_sentences = [[(word, tag) for word, tag in sentence if tag in POS_TAG] for sentence in sentences]

all_idx = set(list(range(len(filter_sentences))))
train_idx=random.sample(all_idx, k=int(np.ceil(len(filter_sentences)*0.8)))
test_idx = all_idx.difference(set(train_idx))

val_idx = random.sample(train_idx, k=int(np.ceil(len(train_idx)*0.1)))
train_idx = set(train_idx).difference(set(val_idx))

train_sentences = [s for idx, s in enumerate(filter_sentences) if idx in train_idx]
val_sentences = [s for idx, s in enumerate(filter_sentences) if idx in val_idx]
test_sentences = [s for idx, s in enumerate(filter_sentences) if idx in test_idx]

- **lexicon preparation: fullform lexicon**

Lexicon is the sole type of feature we used in this model architecture. According to Schmid, lexicon contains "the a priori tag probabilities for each word" (Schmid 1994, pp174). In other word, we can see lexicon as a look-up table, for each word in the lexicon, lexicon stores the probabilities of this word belongs to each part-of-speech tag.

Three types of lexicon were used in this architecture. The first type was *fullform lexicon*. According to Schmid ,the algorithm preparing fullform lexicon was straight forward : "First, the number of occurrences of each word/tag pair was counted. Afterwards, those tags of each word with an estimated probability of less than 1 percent were removed, because they were in most eases the result of tagging errors in the original corpus" (Schmid, pp174). I implemented fullform lexicon in the following cell.


In [246]:
def get_lexicon(g, trim = True):      
    if trim:
        g = g.loc[g['count']>(g['count'].sum()*0.01), :] 
    g['prob'] = g['count']/g['count'].sum()
    return g

def gen_pos_dict(dic):
    pos= dict.fromkeys(POS_TAG,0)
    pos.update(dic)
    return pos

# fullform lexicon prepare
bag = [{'word':word, 'tag':tag} for s in train_sentences for word, tag in s ]
bag = pd.DataFrame(bag)
rev_emmission = bag.groupby('word').apply(lambda g: g.groupby('tag').size()).reset_index(name='count')
full_lexicon = rev_emmission.groupby('word').apply(get_lexicon).reset_index(drop=True)
full_lexicon = full_lexicon.groupby('word').apply(lambda g:gen_pos_dict(dict(zip(g['tag'],g['prob']))))

- **lexicon preparation: suffix lexicon**

The second type of lexicon was *suffix lexicon*, which assigned tag probability to a word not base on the word itself but base on the suffix of the word. To build suffix lexicon, first we had to build a suffix tree which stored all possible suffix form that existed in the words tagged with *open class part-of-speech tag* (i.e., noun, verb, adjective, and adverb). We extracted length one to length five suffixes from of those words. Suffixes were organized into parent-child relations according to the **suffix relation between suffixes**. For example, "s" is the suffix of "es". Therefore node "s" is the direct parent of node "es" in the suffix tree. The root node of suffix tree was an empty string, and all of other suffixes organized their relation following the suffix relation, formed a suffix tree in the end. 

Second, using whole training corpus, we calculated tag frequency for every suffix node in the suffix tree. 

The last step to build suffix tree was pruning. When pruning started, only length-five suffixes were defined as leaf node. **For every leaf node, we calculated it's information gain**, which defined as the information entropy of a leaf node subtracted by its parent node's information entropy, and multiplied the difference by the total frequency of the leaf node. If information gain <10, then the leaf node got deleted. The tag frequency of the deleted leaf node was recollected to a newly created or already existed "default node", which was a leaf node of its original parent node. If after an iteration of pruning, a non-leaf node did not have any child or the only child of a non-leaf node was a default node, the non-leaf node itself became a leaf node and (if existed) the default node got pruned. The pruning process started from the lowest level of the suffix tree (length five suffix) and iteratively processed to the top level (length one suffix), until all leaf nodes were being examined. 

After pruning, we calculated the tag probability for each leaf node. The suffix lexicon was the collection of suffixes' tag probability.

In [184]:
# gen suffix tree nodes
suffix_tree_nodes = pd.Series([word[::-1] for s in train_sentences for word, tag in s 
                               if tag in OPEN_CLASS_POS and len(word)>=5]).unique()
res = []
for i in range(1,6):
    res.append(pd.Series([x[:i] for x in suffix_tree_nodes]).unique())
suffix_tree_nodes = pd.DataFrame(np.concatenate(res), columns=['suffix'])    

In [185]:
# gen suffix tree
suffix_bag = pd.DataFrame([{'suffix':word[::-1][:5],'length':len(word[::-1][:5]),'tag': tag} for s in train_sentences for word, tag in s])
suffix_tree=[]
for i in range(1,6):
    temp=suffix_bag.loc[suffix_bag.length>=i].groupby(
        lambda idx: suffix_bag.loc[idx,'suffix'][:i]).apply(
        lambda g:g.groupby('tag').size()).reset_index(name='count')
    temp=temp.rename({'level_0':'suffix'}, axis=1)
    temp['level']=i
    temp['is_leaf'] = True if i==5 else False
    suffix_tree.append(temp)
suffix_tree = suffix_tree[0].append(suffix_tree[1:])
suffix_tree = suffix_tree.reset_index(drop=True)
suffix_tree = suffix_tree_nodes.merge(suffix_tree, on='suffix')

In [187]:
# prune suffix_tree
def prun(g,level, ent_table, parent_ent=None):
    parent_df  = g.loc[g.level==level,:]
    parent = parent_df.iloc[0,0] if level>0 else ''

    num_children = len(g.loc[g.level==level+1,'suffix'].unique())
    leaf= g.loc[((g.level==level+1) & (g.is_leaf)),:]
    non_leaf = g.loc[((g.level==level+1) & (~g.is_leaf)),:]
    num_leaf = len(g.loc[((g.level==level+1) & (g.is_leaf)),'suffix'].unique())

    # no children
    if num_children==0:        
        parent_df['is_leaf'] = True
        return parent_df
    
    # have at least one leaf
    elif num_leaf>0:
        ## calculating information gain and decide if there is any leaf needs to be pruned
        leaf_freq= leaf.groupby('suffix').apply(lambda g2:g2['count'].sum()).reset_index(name='freq')
        if parent_ent:
            leaf_freq['gain'] = leaf_freq.apply(lambda row: row['freq']*(parent_ent-ent_table[row['suffix']]), axis=1)
        else:            
            leaf_freq['gain'] = leaf_freq.apply(lambda row: row['freq']*(ent_table[parent]-ent_table[row['suffix']]), axis=1)
        prune_leaf = leaf_freq.loc[leaf_freq.gain<10,'suffix']
            
        # all children being drop
        if len(prune_leaf)==num_children:
            parent_df['is_leaf'] = True
            return parent_df
        
        # some leaf have to be prune
        elif len(prune_leaf)>0 and len(prune_leaf)< num_children:
            default = leaf.loc[leaf.suffix.isin(prune_leaf),:].groupby('tag').apply(lambda g:g['count'].sum()).reset_index(name='count')
            default['suffix'] = parent+'?'
            default['level'] = level+1
            default['is_leaf'] = True
            default = default.reindex(['suffix','tag','count','level','is_leaf'] ,axis=1)
            leaf = leaf.loc[~leaf.suffix.isin(prune_leaf),:]
            return parent_df.append([leaf, non_leaf, default])
        
    # all leaf preserved or all children are non_leaf
    return g 

pruned_tree =  suffix_tree
ent = suffix_tree.groupby('suffix').apply(lambda g: entropy(g['count']/g['count'].sum(), base=2))
root_ent = entropy(bag.tag.value_counts(normalize=True).tolist(),base=2)
for i in range(4,-1,-1):
    if i==0:
        pruned = pruned_tree[pruned_tree['level'].isin([i,i+1])].groupby(
                 lambda idx: pruned_tree.loc[idx,'suffix'][:i]).apply(lambda g: prun(g, i, ent, root_ent)).reset_index(drop=True)
    else:
        pruned = pruned_tree[pruned_tree['level'].isin([i,i+1])].groupby(
                 lambda idx: pruned_tree.loc[idx,'suffix'][:i]).apply(lambda g: prun(g,i,ent)).reset_index(drop=True)
    print('finishing pruning for level '+ str(i+1))
    rest = pruned_tree[~pruned_tree['level'].isin([i,i+1])]
    pruned_tree = rest.append(pruned).reset_index(drop=True)

finishing pruning for level 5
finishing pruning for level 4
finishing pruning for level 3
finishing pruning for level 2
finishing pruning for level 1


In [188]:
# gen suffix lexicon 
suffix_lexicon = pruned_tree.loc[pruned_tree.is_leaf,:].groupby('suffix').apply(get_lexicon).reset_index(drop=True)
suffix_lexicon = suffix_lexicon.groupby('suffix').apply(lambda g: gen_pos_dict(dict(zip(g['tag'],g['prob'])))).to_dict()

- **lexicon preparation: default entry**

The last type of lexicon was *default entry*. The tag frequency we used to calculate tag probability of default entry came from the tag frequency of the suffix tree root node subtracted to the sum of the tag frequencies of all leaf node in the suffix tree. 

In [264]:
# defualt entry
leaf_count = pruned_tree.loc[pruned_tree.is_leaf,:].groupby('tag').apply(lambda g:g['count'].sum())
root_count = bag.tag.value_counts()
default_count = root_count-leaf_count.reindex(root_count.index, fill_value=0)
default_count = get_lexicon(pd.DataFrame(default_count, columns=['count']))
default_lexicon = gen_pos_dict(dict(zip(default_count.index, default_count['prob'])))

c:\programdata\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## 2. Transform corpus to the training data

How do we generate training data? First, for every word, the training data generate from a six-words context, which consisted of three previous words, the word itself, and two following words.

After we identified the context for a word, we searched for the correspond tag probabilities for every word in context. Schmid (1994,pp 174) specified the searching process as follows: "During the lookup of a word in the lexicon of the Net-Tagger, the fullform lexicon is searched first. If the word is found there, the corresponding tag probability vector is returned. Otherwise, the uppercase letters of the word are turned to lowercase, and the search in the fullform lexicon is repeated. If it fails again, the suffix lexicon is searched next. If none of the previous steps has been successful, the default entry of the lexicon is returned."

Since every tag probability vector had 36 elements (i.e., 36 kinds of part-of-speech tag) and we returned tag probability vector for all six words in context, every training data was 216 element long.

In [286]:
def word_to_lexicon_feature(word):
    if word is None:
        return dict.fromkeys(POS_TAG,0)
        
    res = {}    
    # try to fetch a priori probability in full_form lexicon
    if word in full_lexicon or word.lower() in full_lexicon:
        #print('from full_lexicon')
        w = word if word in full_lexicon else word.lower()
        res = full_lexicon[w]            
        
    # try to fecth a priori probability in suffix lexicon        
    else:        
        w = word[::-1][:5]
        for i in range(len(w),0,-1):
            tmp_w = w[:i]
            if tmp_w in suffix_lexicon:
                #print('from suffix_lexicon')
                res = suffix_lexicon[tmp_w]
                break
            elif tmp_w[:i-1]+'?' in suffix_lexicon:
                #print('from suffix_lexicon')
                res = suffix_lexicon[tmp_w[:i-1]+'?']
                break
    
    # if the above two approach all failed, return default lexicon
    res = res if res else default_lexicon
    return res

def pick_tag(tags):
    tag_idx = [i for i,t in enumerate(tags) if t is not None]
    return tags[3] if 3 in tag_idx else None

def gen_xy(sentences):
    # making ngram 
    padding =lambda s: ngrams(s, 6, pad_left=True, pad_right=True, left_pad_symbol=(None, None), right_pad_symbol=(None, None))
    data = [words for s in sentences for words in padding(s)]
    
    # mapping data to feature
    words = [[w for w,t in word_tuple] for word_tuple in data]
    tags = [pick_tag([t for w,t in word_tuple]) for word_tuple in data]
    x = [[word_to_lexicon_feature(w) for w in word_list] for idx, word_list in enumerate(words) if tags[idx] is not None]
    y = [t for t in tags if t is not None]
    print('finishing x,y spliting')
    
    # reshape
    dv= DictVectorizer(sparse=False).fit([dict.fromkeys(POS_TAG,0)])
    x = [[dv.fit_transform(dic)[0] for dic in w] for w in x]
    x = np.vstack([np.concatenate(w, axis=0) for w in x])
    print('finishing x transformation')
    y = np.vstack([dv.fit_transform(gen_pos_dict({str(t):1})) for t in y])
    print('finishing y transformation')
    return x, y
    

In [270]:
train_x, train_y = gen_xy(train_sentences)
val_x, val_y = gen_xy(val_sentences)

finihsing x,y spliting
finishing x transformation
finishing y trnasformation
finihsing x,y spliting
finishing x transformation
finishing y trnasformation


## 3. Model Specification

I replicated the original model and training architecture design when it was possible. The architecture was a full-connected neural network with no hidden layer. The Input layer activated by sigmoid function and the output layer activated by softmax function. It was the same architecture I used here. The other training parameters, like epoch, batch size...etc, did not mention in the original paper. Therefore I assigned those training parameter by myself.

In [281]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier

def build_model(input_dim, hidden_neurons, output_dim):
    model = Sequential([
        Dense(hidden_neurons, input_dim=input_dim),
        Activation('sigmoid'),
        Dense(output_dim, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_params = {
    'build_fn': build_model,
    'input_dim': train_x.shape[1],
    'hidden_neurons': 512,
    'output_dim': train_y.shape[1],
    'epochs': 16,
    'batch_size': 64,
    'verbose': 1,
    'validation_data': (val_x, val_y),
    'shuffle': True
}

clf = KerasClassifier(**model_params)

## Result and Discussion

After training, we found that the part-of-speech predicting accuracy on training data was about 97%, on validation data was about 96.7%, and on testing data was about 96.3%. Since the model performance on training and testing were pretty similar, we can say that there was almost no overfitting problem.

The replicated model seemly outperformed the original model. The original model had about 92% accuracy with 60K words training corpus. However, the performance difference may come from the original model**didn't filtered out the words that were tagged with a non-part-of-speech tag**. In other words, the performance difference of replicated and original model may come from that the original model built and tested on a more "impure" corpus, which made it harder to produce the right part-of-speech tag.

In [290]:
hist = clf.fit(train_x, train_y)

Train on 59273 samples, validate on 6421 samples
Epoch 1/16
59273/59273 [==============================] - 8s 134us/step - loss: 0.8211 - acc: 0.8317 - val_loss: 0.2069 - val_acc: 0.9528
Epoch 2/16
59273/59273 [==============================] - 6s 97us/step - loss: 0.1676 - acc: 0.9573 - val_loss: 0.1508 - val_acc: 0.9601
Epoch 3/16
59273/59273 [==============================] - 6s 96us/step - loss: 0.1384 - acc: 0.9614 - val_loss: 0.1403 - val_acc: 0.9603
Epoch 4/16
59273/59273 [==============================] - 6s 104us/step - loss: 0.1286 - acc: 0.9639 - val_loss: 0.1345 - val_acc: 0.9634
Epoch 5/16
59273/59273 [==============================] - 6s 105us/step - loss: 0.1218 - acc: 0.9653 - val_loss: 0.1305 - val_acc: 0.9626
Epoch 6/16
59273/59273 [==============================] - 6s 98us/step - loss: 0.1184 - acc: 0.9662 - val_loss: 0.1315 - val_acc: 0.9639
Epoch 7/16
59273/59273 [==============================] - 6s 99us/step - loss: 0.1150 - acc: 0.9668 - val_loss: 0.1272 - val_a

In [287]:
test_x, test_y = gen_xy(test_sentences)

finishing x,y spliting
finishing x transformation
finishing y transformation


In [291]:
score = clf.score(test_x, test_y)
print(score)

16675/16675 [==============================] - 1s 41us/step
0.9632383808131697


## References

Helmut Schmid. 1994. Part-of-speech tagging with neural networks. In Proceedings of the 15th conference on Computational linguistics - Volume 1 (COLING '94), Vol. 1. Association for Computational Linguistics, Stroudsburg, PA, USA, 172-176. DOI: https://doi.org/10.3115/991886.991915 